In [ ]:
import polars as pl
import numpy as np
# from utils.process_data import process_item_data, explode_user_interactions TODO see if I need this
from parquet_data_reader import ParquetDataReader
from models.user_based import CollaborativeReccomender

pl.Config.set_tbl_cols(-1)

## Data import and EDA

## Preprocessing

## Model Fit

## Model presentation